In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from IPython.display import clear_output, Image, display, HTML

###### Do not modify here ###### 
def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = graph_def
    #strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))
###### Do not modify  here ######

###### Implement Data Preprocess here ######
housing = fetch_california_housing()
# Show description/ statisics about the dataset
print(housing.DESCR)

California housing dataset.

The original database is available from StatLib

    http://lib.stat.cmu.edu/

The data contains 20,640 observations on 9 variables.

This dataset contains the average house value as target variable
and the following input variables (features): average income,
housing average age, average rooms, average bedrooms, population,
average occupation, latitude, and longitude in that order.

References
----------

Pace, R. Kelley and Ronald Barry, Sparse Spatial Autoregressions,
Statistics and Probability Letters, 33 (1997) 291-297.




## Data Pre-processing

In [2]:
# Define cut point
cut = int(housing.data.shape[0]*0.9)
# Convert to column vector format, which is (n,1) not (n,)
housing.target = housing.target.reshape(housing.target.shape[0],1)
# Split dataset
x_train, x_test = housing.data[:cut], housing.data[cut:]
y_train, y_test = housing.target[:cut], housing.target[cut:]

print("\nShape of training set:", x_train.shape)
print("Shape of testing set:", x_test.shape)
print("Shape of training label:", y_train.shape)
print("Shape of testing label:", y_test.shape)


Shape of training set: (18576, 8)
Shape of testing set: (2064, 8)
Shape of training label: (18576, 1)
Shape of testing label: (2064, 1)


### Standardization

In [16]:
# x_train_ = np.concatenate((x_train,np.ones([x_train.shape[0],1])),axis=1)
# x_test_  = np.concatenate((x_test ,np.ones([ x_test.shape[0],1])),axis=1)
# x_train = np.divide(x_train - np.mean(x_train, axis=0, keepdims=True), np.std(x_train, axis=0, keepdims=True))
# x_test  = np.divide(x_test  - np.mean(x_train, axis=0, keepdims=True), np.std(x_train, axis=0, keepdims=True))
x_train_ = np.concatenate((np.divide(x_train - np.mean(x_train, axis=0, keepdims=True), np.std(x_train, axis=0, keepdims=True)),np.ones([x_train.shape[0],1])),axis=1)
x_test_  = np.concatenate((np.divide(x_test  - np.mean(x_train, axis=0, keepdims=True), np.std(x_train, axis=0, keepdims=True)),np.ones([ x_test.shape[0],1])),axis=1)

## Define Graph

In [17]:
# Clear graph
tf.reset_default_graph()

# Define variables to take input feature x, label y
x = tf.placeholder(tf.float64, shape = [None, x_train_.shape[1]], name="Input")
y = tf.placeholder(tf.float64, shape = [None, 1], name="Y")
W = tf.placeholder(tf.float64, shape = [9, 1], name="W")

# Calculate W
XT = tf.matrix_transpose(x, name="X_Transpose") # calculate transpose x
W_train = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, x, name="XT_X") , name="Inverse_XT_X"),XT, name="Multiple_XT"),y, name="Multiple_y") # calculate w by formula

# Systems of linear equation
linear_regression = tf.matmul(x, W, name="Linear_Regression")

# Calculate the mean of error rate
mean_error_rate = tf.reduce_mean(tf.divide(tf.abs(tf.subtract(y,linear_regression, name="Predict_Difference")),y,name="Error_Rate"), name="Mean_Error_Rate")


### Graph Explain and Result

To get the weight $W$ for linear equation 

$\hat y = XW + b$

We can utilizing following equation to solve it

$W=(X^{(train)\top} X^{(train)})^{-1} X^{(train)\top} y^{(train)}$

1. We set the placeholder to take the input feature as variable x and label as variable y, which corresponding to **Input** and **Y** in the graph below. Note that the x is original feature concatenate with 1 which represented the Bias $b$.

2. To calculate the Weight $W$,

    a. We calculate the element transpose X as variable XT which is **Inverse_XT_X** in graph.
    
    b. Calculate though the equation $W=(X^{(train)\top} X^{(train)})^{-1} X^{(train)\top} y^{(train)}$ to get W.
    
    c. By this step, we already get W for this systems of linear equation. Therefore, we are able to calculate the predict y via $XW$ as **Linear_Regression** in the graph.
    
    d. Calculte the mean error rate via $ \frac{1}{m} \sum \frac{\lvert(y -\hat y) \rvert}{y}$

In [15]:
init_w = np.ones((9,1),dtype=np.float64)
with tf.Session() as sess:
    ###### Start TF session ######
    W_t = sess.run(W_train, {x: x_train_, y: y_train, W:init_w}) # Calculate W
    print(W_t)
    sess.run(tf.global_variables_initializer())
    print("Training Error Rate: ", sess.run(mean_error_rate, {x: x_train_, y: y_train, W:W_t}))
    print( "Testing Error Rate: ", sess.run(mean_error_rate, {x: x_test_, y: y_test, W:W_t}))
    
    show_graph(tf.get_default_graph().as_graph_def())

[[  4.42440871e-01]
 [  9.84845976e-03]
 [ -1.13854700e-01]
 [  6.81118426e-01]
 [ -1.69407374e-06]
 [ -5.61851833e-03]
 [ -4.31983935e-01]
 [ -4.46590962e-01]
 [ -3.80329231e+01]]
Training Error Rate:  0.316859910108
Testing Error Rate:  0.344205405992


In [21]:
init_w = np.ones((9,1),dtype=np.float64)
with tf.Session() as sess:
    ###### Start TF session ######
    W_t = sess.run(W_train, {x: x_train_, y: y_train, W:init_w}) # Calculate W
    print(W_t)
    sess.run(tf.global_variables_initializer())
    print("Training Error Rate: ", sess.run(mean_error_rate, {x: x_train_, y: y_train, W:W_t}))
    print( "Testing Error Rate: ", sess.run(mean_error_rate, {x: x_test_, y: y_test, W:W_t}))
    
    show_graph(tf.get_default_graph().as_graph_def())

[[  8.55045942e-01]
 [  1.24666428e-01]
 [ -2.89596488e-01]
 [  3.30602843e-01]
 [ -1.93507967e-03]
 [ -3.41520988e-02]
 [ -8.90462336e-01]
 [ -8.90240609e-01]
 [  2.12031204e+00]]
Training Error Rate:  0.316859910109
Testing Error Rate:  0.344205405993
